In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

np.random.seed(24680)

Write your imports in the cell below.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC

# Ensemble Models and Support Vector Machines Lab
## Training and comparing different algorithms

Once again, we'll work with the bank dataset. This time, the data preprocessing steps have been done for you.

The goal is to try and improve our predictions (if they can be improved at all) using different types of algorithms.

### 1. Read the data (1 point)
This time you only need to read the data. The indicator variables have been separated out for you.

Read the dataset and save it in the variable `bank_data`. The target column is `y`. Use the variables `bank_attributes` and `bank_labels` to save the attributes (explanatory variables, features, predictors), and labels (`y`).

In [4]:
bank_data = None
bank_attributes = None
bank_labels = None
bank_data = pd.read_csv("data/bank.csv")
bank_data

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,33,4789,11,220,1,339,4,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,59,0,5,226,1,-1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,-333,30,329,5,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4517,57,-3313,9,153,1,-1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4518,57,295,19,151,11,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4519,28,1137,6,129,4,211,3,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
bank_attributes = bank_data.drop("y", axis = 1)
bank_labels = bank_data.y

In [6]:
assert_is_not_none(bank_data)
assert_is_not_none(bank_attributes)
assert_is_not_none(bank_labels)

### 2. Normalize the data (1 point)
Because both forests and SVMs are sensitive to non-scaled data, we need to normalize our dataset first.

Rescale all columns in `bank_attributes` so they have mean = 0 and variance = 1. You can either look at the `sklearn` docs or do this yourself. When you're ready, overwrite the `bank_attributes` column. Make sure that you don't lose the column names in the process.

In [7]:
scaler = StandardScaler()
#Official Documentation: The standard score of a sample x is calculated as: z = (x - u) / s
# where u is the mean of the training samples or zero if with_mean=False, and s is the standard deviation of the training 
# samples or one if with_std=False.
#Another article: https://medium.com/towards-artificial-intelligence/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff
# standardization (or Z-score normalization) means centering the variable at zero and standardizing the variance at 1.
scaled_attributes = scaler.fit_transform(bank_attributes)
scaled_attributes

array([[-1.05626965,  0.12107186,  0.37405206, ..., -0.21344711,
        -0.1713814 ,  0.46930045],
       [-0.77258281,  1.1186443 , -0.59602646, ..., -0.21344711,
        -0.1713814 , -2.1308311 ],
       [-0.58345826, -0.02414438,  0.01027262, ..., -0.21344711,
        -0.1713814 , -2.1308311 ],
       ...,
       [ 1.49691189, -0.37472364,  0.37405206, ..., -0.21344711,
        -0.1713814 ,  0.46930045],
       [-1.24539421, -0.09492484, -1.20232553, ...,  4.68500145,
        -0.1713814 , -2.1308311 ],
       [ 0.26760226, -0.09525714, -1.56610497, ...,  4.68500145,
        -0.1713814 , -2.1308311 ]])

In [8]:
bank_attributes = pd.DataFrame(scaled_attributes, index=bank_attributes.index, columns=bank_attributes.columns)
bank_attributes

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,-1.056270,0.121072,0.374052,-0.711861,-0.576829,-0.407218,-0.320413,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,-0.669064,-0.306828,7.450671,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
1,-0.772583,1.118644,-0.596026,-0.169194,-0.576829,2.989044,2.041734,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,1.494626,-0.306828,-0.134216,-0.107869,2.868193,-0.213447,-0.171381,-2.130831
2,-0.583458,-0.024144,0.010273,-0.303898,-0.576829,2.899143,0.270124,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,-0.669064,-0.306828,-0.134216,-0.107869,2.868193,-0.213447,-0.171381,-2.130831
3,-1.056270,0.017726,-1.566105,-0.250017,0.387967,-0.407218,-0.320413,-0.343844,-0.514408,-0.196454,...,2.741190,-0.104676,-0.669064,-0.306828,-0.134216,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
4,1.686036,-0.472753,-1.323585,-0.146102,-0.576829,-0.407218,-0.320413,-0.343844,1.943983,-0.196454,...,-0.364805,-0.104676,1.494626,-0.306828,-0.134216,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,-0.772583,-0.583410,1.707910,0.250315,0.709566,-0.407218,-0.320413,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,-0.669064,-0.306828,-0.134216,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
4517,1.496912,-1.573671,-0.838546,-0.427057,-0.576829,-0.407218,-0.320413,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,1.494626,-0.306828,-0.134216,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
4518,1.496912,-0.374724,0.374052,-0.434754,2.639160,-0.407218,-0.320413,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,-0.669064,-0.306828,-0.134216,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
4519,-1.245394,-0.094925,-1.202326,-0.519426,0.387967,1.710451,1.451197,-0.343844,1.943983,-0.196454,...,-0.364805,-0.104676,-0.669064,-0.306828,-0.134216,-0.107869,-0.348652,4.685001,-0.171381,-2.130831


In [9]:
print(scaled_attributes.mean(axis=0))
print(scaled_attributes.std(axis=0))

[-1.17873712e-16 -1.57164949e-17  2.35747424e-18 -7.62250004e-17
 -6.28659797e-18 -3.88983250e-17  3.92912373e-18 -2.31818300e-17
 -3.69337631e-17  4.08628868e-17  4.40061858e-17 -1.96456187e-17
 -1.57164949e-17  1.10015465e-17 -1.41448454e-17 -5.50077323e-18
  1.33590207e-17  2.84861471e-17  1.41448454e-17 -4.67565724e-17
  7.85824746e-17  6.28659797e-18 -1.57164949e-18 -1.25731959e-17
  1.21802836e-16  1.88597939e-17 -2.75038661e-18  2.75038661e-18
 -7.97612118e-17  1.43413016e-16 -7.85824746e-17  5.97226807e-17
 -3.14329899e-18 -1.10015465e-17  2.67180414e-17  0.00000000e+00
 -3.14329899e-18 -1.57164949e-18  2.35747424e-17  4.08628868e-17
 -6.28659797e-18 -7.85824746e-18 -9.42989696e-18 -7.70108252e-17
  3.61479383e-17  1.80739692e-17 -1.13944588e-17  4.87211343e-17
 -1.17873712e-17  1.57164949e-18 -1.04121779e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]


In [10]:
assert_is_not_none(bank_attributes)

### 3. Split the data (1 point)
Use the standard 70% / 30% split. Since this is a classification problem, be sure to stratify the split according to the `bank_labels`.

In [11]:
bank_attributes_train, bank_labels_train = None, None
bank_attributes_test, bank_labels_test = None, None
bank_attributes_train, bank_attributes_test, bank_labels_train, bank_labels_test = train_test_split(
    bank_attributes, bank_labels, train_size = 0.7, stratify = bank_labels)

In [12]:
assert_is_not_none(bank_attributes_train)
assert_is_not_none(bank_labels_train)

assert_is_not_none(bank_attributes_test)
assert_is_not_none(bank_labels_test)

### 4. Prepare the cross-validation folds (1 point)
Use a stratified k-fold cross-validation split, with $k = 5$. Fit it to the train data. Save the trained cross-validator to the variable `k_fold`.

The data should already be shuffled. There's no need to shuffle it again.

In [13]:
k_fold = None
k_fold = StratifiedKFold(n_splits = 5)
k_fold

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [14]:
assert_is_not_none(k_fold)

### 5. Decision Tree (2 points)
Use cross-validation to train and optimize the hyperparameters for a decision tree classifier.

Use grid search with the following grid:
* `max_depth`: 1, 5, 7, 15, 20
* `min_samples_leaf`: 2, 5, 10, 12
* `max_leaf_nodes`: 5, 10, 20

Use the most appropriate scoring metric (remember that accuracy doesn't work in this case because the data is highly imbalanced; we need something which combines precision and recall). Use the cross-validation splits you just created.

Save the grid results in `grid_search`. Save the best classifier in `tree_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [15]:
grid_search = None
tree_classifier = None

parametersDT = {
    "max_depth": [1, 5, 7, 15, 20],
    "min_samples_leaf": [2, 5, 10, 12],
    "max_leaf_nodes": [5, 10, 20]
}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid = parametersDT, cv = k_fold, scoring = 'f1')
grid_search.fit(bank_attributes_train, bank_labels_train)

tree_classifier = grid_search.best_estimator_

print("Decision tree; best score:", grid_search.best_score_)

Decision tree; best score: 0.5090152112059995


In [16]:
tree_classifier

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=15, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [17]:
assert_is_not_none(grid_search)
assert_is_not_none(tree_classifier)

### 5. Random Forest (1 point)
Use cross-validation to train and optimize the hyperparameters for a random forest classifier. Use the same technique as before.

Use the following grid:
* `n_estimators`: 100, 200, 300 
* `max_depth`: 20, 50, 100

Note that this grid is on the small side but this is mainly due to performance reasons. Also note that the training will take some time.

Save the grid results in `grid_search`. Save the best classifier in `forest_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

Due to the relatively slow training, we've chosen low values for the parameters. The performance of the random forest will be worse than the decision tree. This is not necessarily the case in general, it's due to the parameters we've chosen to try.

In [18]:
grid_search = None
forest_classifier = None

parametersRF = {
    "n_estimators": [100, 200, 300],
    "max_depth": [20, 50, 100],
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid = parametersRF, cv = k_fold, scoring = 'f1')
grid_search.fit(bank_attributes_train, bank_labels_train)

forest_classifier = grid_search.best_estimator_

print("Random forest; best score:", grid_search.best_score_)

Random forest; best score: 0.39249980844504256


In [19]:
assert_is_not_none(grid_search)
assert_is_not_none(forest_classifier)

### 6. Linear SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for a linear support vector machine. Use the same technique as before.

Use the following grid:
* `C`: 0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20

Note that we're choosing relatively small values for `C`. This is allowed because our data is normalized.

Save the grid results in `grid_search`. Save the best classifier in `linear_svm_classifier`. There are many ways to create a linear SVM classifier. Look up the `sklearn` docs to choose the fastest one (in terms of performance).

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [20]:
grid_search = None
linear_svm_classifier = None

parametersLSVM = {
    "C": [0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20]
}
grid_search = GridSearchCV(LinearSVC(), param_grid = parametersLSVM, cv = k_fold, scoring = 'f1')
grid_search.fit(bank_attributes_train, bank_labels_train)

linear_svm_classifier = grid_search.best_estimator_

print("Linear SVM; best score:", grid_search.best_score_)

C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the numb

C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the numb

Linear SVM; best score: 0.41866870906870907


C:\Users\PC-Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [21]:
assert_is_not_none(grid_search)
assert_is_not_none(linear_svm_classifier)

### 7. Gaussian SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for an SVM with a Gaussian kernel. Use the same technique as before.

Use the following grid:
* `C`: 10, 15, 20, 50, 200
* `gamma`: 0.001, 0.01, 0.1, 0.2

Note that this time we give larger values of `C` because the governing parameter here is `gamma`.

Save the grid results in `grid_search`. Save the best classifier in `gaussian_svm_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [22]:
grid_search = None
gaussian_svm_classifier = None

parametersGSVM = {
    "C": [10, 15, 20, 50, 200],
    "gamma": [0.001, 0.01, 0.1, 0.2]
}
grid_search = GridSearchCV(SVC(), param_grid = parametersGSVM, cv = k_fold, scoring = 'f1')
grid_search.fit(bank_attributes_train, bank_labels_train)

gaussian_svm_classifier = grid_search.best_estimator_

print("Gaussian SVM; best score:", grid_search.best_score_)

Gaussian SVM; best score: 0.45760736147055053


In [23]:
assert_is_not_none(grid_search)
assert_is_not_none(gaussian_svm_classifier)

### 9. Compare performance on the testing data (1 point)
Now that you've trained all your models, you've got to select the best one. This should be done on the testing data.

Use the appropriate scoring metric to get the testing scores for all your models. Don't forget to pass the **testing**, not the training data. Save all scores.

Choose the best classifier, based on these scores (the one with the highest test score). Of course, this is not enough. We need to look at ROC curves, track performance through other measures, debug the sources of variance in testing results, try more hyperparameters, etc. However, this is enough for an introductory lab :).

Optionally, you can think of combining them into a boosted model but this is out of the scope of this lab.

In [24]:
tree_classifier_score, forest_classifier_score, linear_svm_classifier_score, gaussian_svm_classifier_score = None, None, None, None

tree_classifier_score = tree_classifier.score(bank_attributes_test, bank_labels_test)
forest_classifier_score = forest_classifier.score(bank_attributes_test, bank_labels_test)
linear_svm_classifier_score = linear_svm_classifier.score(bank_attributes_test, bank_labels_test)
gaussian_svm_classifier_score = gaussian_svm_classifier.score(bank_attributes_test, bank_labels_test)

print("Testing scores:")
print("Decision tree:", tree_classifier_score)
print("Random forest:", forest_classifier_score)
print("Linear SVM:", linear_svm_classifier_score)
print("Gaussian SVM:", gaussian_svm_classifier_score)

Testing scores:
Decision tree: 0.8953574060427414
Random forest: 0.8953574060427414
Linear SVM: 0.8739867354458364
Gaussian SVM: 0.8887251289609432


In [25]:
best_classifier = "" # Replace empty string with "tree", "forest", "linear SVM" or "gaussian SVM"

dictionary_of_scores = {
                  tree_classifier_score:"tree",
                  forest_classifier_score:"forest",
                  linear_svm_classifier_score:"linear SVM",
                  gaussian_svm_classifier_score:"gaussian SVM"
}

best_classifier = dictionary_of_scores.get(max(tree_classifier_score, forest_classifier_score, linear_svm_classifier_score, gaussian_svm_classifier_score))
best_classifier

'forest'

In [26]:
assert_not_equal(best_classifier, "")